In [1]:
import numpy as np
from pathlib import Path
olderr = np.seterr(divide='ignore')

In [2]:
from box import Box, DataSet
from units import unit

In [3]:
# settings

data_folder = Path("/mnt/Knolselderij/bulk/Abrupt/")
month = 3

sigma_d = unit('200 km')
sigma_t = unit('5 year')

gamma = 1
scaling_factor = gamma * unit('10 km/year')
sobel_delta_t = unit('1 year')
sobel_delta_d = sobel_delta_t * scaling_factor
sobel_weights = [sobel_delta_t, sobel_delta_d, sobel_delta_d]

In [4]:
control_set = DataSet(
    path=data_folder,
    model='MPI-ESM-LR',
    variable='tas',
    scenario='piControl',
    realization='r1i1p1')
control_set.load()

In [5]:
control_data = control_set.data[month-1::12]
control_box = control_set.box[month-1::12]
smooth_control_data = control_box.gaussian_filter(control_data, [sigma_t, sigma_d, sigma_d])

In [7]:
cal = control_box.calibrate_sobel(smooth_control_data, sobel_delta_t, sobel_delta_d)

In [11]:
for k, v in cal.items():
    print("{:10}: {}".format(k, v))
print("recommended setting for gamma: ", cal['gamma'][3])

time      : [ 0.          0.01403261  0.0314737   0.0623523   1.15789974]
distance  : [  1.01828191e-05   5.16311750e-02   9.72578079e-02   1.58531472e-01
   1.30268240e+00]
magnitude : [  2.02006049e-04   6.32240772e-02   1.08935483e-01   1.80317283e-01
   1.31090987e+00]
gamma     : [ 0.          0.2717855   0.32361102  0.39331177  0.88885802]
recommended setting for gamma:  0.393312
